Preprocessing of data to create training and testing sets of data

In [332]:
import pandas as pd
import numpy as np
from ast import literal_eval

import warnings; warnings.simplefilter('ignore')

In [333]:
#read in data from csv to dataframes

movies_metadata_df = pd.read_csv("TheMoviesDataset\movies_metadata.csv")
ratings_df = pd.read_csv("TheMoviesDataset/ratings_small.csv") #used / instead of \ due to '\r' representing carriage return in string casusing a error
credits_df = pd.read_csv("TheMoviesDataset/credits.csv")
keywords_df = pd.read_csv("TheMoviesDataset/keywords.csv")

In [334]:
print('movies: ', movies_metadata_df.shape)
print('ratings: ', ratings_df.shape)
print('credits: ', credits_df.shape)
print('keywords: ', keywords_df.shape)

movies:  (45466, 24)
ratings:  (100004, 4)
credits:  (45476, 3)
keywords:  (46419, 2)


In [335]:
movies_metadata_df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [336]:
ratings_df.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [337]:
credits_df.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [338]:
keywords_df.head(5)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [339]:
#rename 'id' to 'movieId' in movie_metadata, credits, and keywords to be consitant across data

movies_metadata_df = movies_metadata_df.rename(columns={'id': 'movieId'})
credits_df = credits_df.rename(columns={'id': 'movieId'})
keywords_df = keywords_df.rename(columns={'id': 'movieId'})

In [340]:
#incorrectly formated data found within dataset, easier to drop movies by index than fix

movies_metadata_df = movies_metadata_df.drop(35586)
movies_metadata_df = movies_metadata_df.drop(35587)
movies_metadata_df = movies_metadata_df.drop(29502)
movies_metadata_df = movies_metadata_df.drop(29503)
movies_metadata_df = movies_metadata_df.drop(19729)
movies_metadata_df = movies_metadata_df.drop(19730)

#reset index so that theres none missing
movies_metadata_df.reset_index(drop=True, inplace=True)

In [341]:
#change 'movieId' type to 'int'

movies_metadata_df['movieId'] = movies_metadata_df['movieId'].astype('int')
credits_df['movieId'] = credits_df['movieId'].astype('int')
keywords_df['movieId'] = keywords_df['movieId'].astype('int')

re order later use this some how as you're using .apply(literal_eval) alot

In [342]:
# from ast import literal_eval
# features = ['cast', 'crew', 'genres','keywords']
# for feature in features:
#     df[feature] = df[feature].apply(literal_eval)

In [343]:
movies_metadata_df['genres'] = movies_metadata_df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies_metadata_df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [344]:
#Transform keywords from [{'id': 0, 'name': 'x'}, {'id': 1, 'name': 'y'} to [x, y]

keywords_df['keywords'] = keywords_df['keywords'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
keywords_df.head(5)

,movieId,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."


In [323]:
credits_df['cast'] = credits_df['cast'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
credits_df.head(5)

,cast,crew,movieId
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [324]:

credits_df['crew'] = credits_df['crew'].fillna('[]').apply(literal_eval)#.apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
credits_df.head(5)

,cast,crew,movieId
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [325]:
filtered_df = credits_df[credits_df['movieId']==15602]
for x in filtered_df['crew']:
    print(x)


[{'credit_id': '52fe466a9251416c75077a89', 'department': 'Directing', 'gender': 2, 'id': 26502, 'job': 'Director', 'name': 'Howard Deutch', 'profile_path': '/68Vae1HkU1NxQZ6KEmuxIpno7c9.jpg'}, {'credit_id': '52fe466b9251416c75077aa3', 'department': 'Writing', 'gender': 2, 'id': 16837, 'job': 'Characters', 'name': 'Mark Steven Johnson', 'profile_path': '/6trChNn3o2bi4i2ipgMEAytwmZp.jpg'}, {'credit_id': '52fe466b9251416c75077aa9', 'department': 'Writing', 'gender': 2, 'id': 16837, 'job': 'Writer', 'name': 'Mark Steven Johnson', 'profile_path': '/6trChNn3o2bi4i2ipgMEAytwmZp.jpg'}, {'credit_id': '5675eb4b92514179dd003933', 'department': 'Crew', 'gender': 2, 'id': 1551320, 'job': 'Sound Recordist', 'name': 'Jack Keller', 'profile_path': None}]


In [326]:
#getting director(s) from crew, if no director(s) return empty list

def get_directors(x):
    return [i['name'] for i in x if i['job'] == 'Director']

#getting writer(s) from crew, if no writer(s) return empty list

def get_writers(x):
    return [i['name'] for i in x if i['job'] in ['Screenplay', 'Writer']]

#apply functions

credits_df['directors'] = credits_df['crew'].apply(get_directors)
credits_df['writers'] = credits_df['crew'].apply(get_writers)

credits_df.head(5)

,cast,crew,movieId,directors,writers
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,[John Lasseter],"[Joss Whedon, Andrew Stanton, Joel Cohen, Alec..."
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,[Joe Johnston],"[Jonathan Hensleigh, Greg Taylor, Jim Strain]"
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch],[Mark Steven Johnson]
3,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,[Forest Whitaker],"[Ronald Bass, Terry McMillan]"
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,[Charles Shyer],"[Nancy Meyers, Albert Hackett]"


In [327]:
credits_df = credits_df.drop('crew', axis=1)
credits_df.head(5)

,cast,movieId,directors,writers
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",862,[John Lasseter],"[Joss Whedon, Andrew Stanton, Joel Cohen, Alec..."
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",8844,[Joe Johnston],"[Jonathan Hensleigh, Greg Taylor, Jim Strain]"
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",15602,[Howard Deutch],[Mark Steven Johnson]
3,"[Whitney Houston, Angela Bassett, Loretta Devi...",31357,[Forest Whitaker],"[Ronald Bass, Terry McMillan]"
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...",11862,[Charles Shyer],"[Nancy Meyers, Albert Hackett]"


In [328]:
credits_df = credits_df.reindex(columns=['movieId','directors','writers','cast'])
credits_df.head(5)

,movieId,directors,writers,cast
0,862,[John Lasseter],"[Joss Whedon, Andrew Stanton, Joel Cohen, Alec...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,8844,[Joe Johnston],"[Jonathan Hensleigh, Greg Taylor, Jim Strain]","[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,15602,[Howard Deutch],[Mark Steven Johnson],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,31357,[Forest Whitaker],"[Ronald Bass, Terry McMillan]","[Whitney Houston, Angela Bassett, Loretta Devi..."
4,11862,[Charles Shyer],"[Nancy Meyers, Albert Hackett]","[Steve Martin, Diane Keaton, Martin Short, Kim..."


In [329]:
movies_features_df = pd.merge(credits_df, movies_metadata_df[['movieId','genres']], on='movieId')
movies_features_df.head(5)

,movieId,directors,writers,cast,genres
0,862,[John Lasseter],"[Joss Whedon, Andrew Stanton, Joel Cohen, Alec...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Animation, Comedy, Family]"
1,8844,[Joe Johnston],"[Jonathan Hensleigh, Greg Taylor, Jim Strain]","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Adventure, Fantasy, Family]"
2,15602,[Howard Deutch],[Mark Steven Johnson],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Romance, Comedy]"
3,31357,[Forest Whitaker],"[Ronald Bass, Terry McMillan]","[Whitney Houston, Angela Bassett, Loretta Devi...","[Comedy, Drama, Romance]"
4,11862,[Charles Shyer],"[Nancy Meyers, Albert Hackett]","[Steve Martin, Diane Keaton, Martin Short, Kim...",[Comedy]


In [330]:
# function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    return [str.lower(i.replace(' ', '')) for i in x]

# list of features to clean
features = ['directors', 'writers', 'cast', 'genres']

# apply function to each feature using map() instead of apply()
movies_features_df[features] = movies_features_df[features].applymap(clean_data)

In [331]:
movies_features_df.to_csv('movies_features.csv', index=False)